You should remove the `raise` exceptions below and insert your code in their place. The cells which say `DO NOT CHANGE THE CONTENT OF THIS CELL` are there to help you, if they fail, it's probably an indication of the fact that your code is wrong. You should not change their content - if you change them to make them correspond to what your program is producing, you will still not get the marks.

Before you turn your solution in, make sure everything runs as expected. With an attached cluster, you should **Clear State and Results** (under the **Clear** dropdown menu) and then click on the **Run all** icon. This runs all cells in the notebook from new. You should only submit this notebook if all cells run.

# Scrubbing data

A common part of the ETL process is data scrubbing. This homework asks you to process data in order to get it into a standardized format for later processing.

The file `devicestatus.txt.zip` is available from Blackboard. This file contains data collected from mobile devices on a network, including device ID, current status, location and so on. Because the company previously acquired other mobile provider's networks, the data from different subnetworks has a different format. Note that the records in this file have different field delimiters: some use commas, some use pipes (|), and so on. 

This notebook will guide you through the steps of scrubbing this dataset. Follow the instructions carefully. In general, every time you execute a step, there is a check that tests whether you carried out that step correctly. You should not move on until a step is passing the test. All your processing should be done in this notebook. It is assumed that you have uncompressed the `devicestatus.txt` file and have placed it in `/FileStore/` on DBFS using the steps described in the labs this week.

The first two cells perform some checks: first one checks that you are running this notebook on Databricks, and the second cell ensures that your file is starting off in `/FileStore/devicestatus.txt` You should not (try to) change either of these cells.

In [0]:
# DO NOT CHANGE THE CONTENT OF THIS CELL
import imp

try:
    imp.find_module('dbutils')
except ImportError:
    import pyspark
    sc = pyspark.SparkContext()

In [0]:
# DO NOT CHANGE THE CONTENT OF THIS CELL
def file_exists(path):
    try:
        dbutils.fs.ls(path)
        return True
    except Exception as e:
        if 'java.io.FileNotFoundException' in str(e):
            return False
        else:
            raise

try:
    imp.find_module('dbutils')     
    assert file_exists("/FileStore/devicestatus.txt") == True, "It is assumed that you've put your (unziped) devicestatus.txt file in the DBFS /FileStore"
except ImportError:
    print("Are you running this on Databricks?")

Load the `devicestatus.txt` dataset into a variable called `myRDD`.

In [0]:
# Read the devicestatus.txt file into an RDD. Your answer should have the following format (without comment tag)

myRDD = sc.textFile("dbfs:/FileStore/devicestatus.txt")

#raise NotImplementedError()

In [0]:
# DO NOT CHANGE THE CONTENT OF THIS CELL
assert myRDD.count() == 459540, "It doesn't look like you've ready in your data correctly"

Determine which delimiter(s) to use and divide up each line of the RDD into a list of its fields. Your result should be in a variale called `splitRDD`.

In [0]:
myRDD.take(5)

Out[87]: ['2014-03-15:10:10:20,Sorrento F41L,8cc3b47e-bd01-4482-b500-28f2342679af,7,24,39,enabled,disabled,connected,55,67,12,33.6894754264,-117.543308253',
 '2014-03-15:10:10:20|MeeToo 1.0|ef8c7564-0a1a-4650-a655-c8bbd5f8f943|0|31|63|70|39|27|enabled|enabled|enabled|37.4321088904|-121.485029632',
 '2014-03-15:10:10:20|MeeToo 1.0|23eba027-b95a-4729-9a4b-a3cca51c5548|0|20|21|86|54|34|enabled|enabled|enabled|39.4378908349|-120.938978486',
 '2014-03-15:10:10:20,Sorrento F41L,707daba1-5640-4d60-a6d9-1d6fa0645be0,8,22,60,enabled,enabled,disabled,68,91,17,39.3635186767,-119.400334708',
 '2014-03-15:10:10:20,Ronin Novelty Note 1,db66fe81-aa55-43b4-9418-fc6e7a00f891,0,13,47,70,enabled,enabled,enabled,10,45,33.1913581092,-116.448242643']

In [0]:
# The format of your answer should be

# PROBLEM - Types of delimiters present - "/" , "|" , "," 
# ACTION TAKEN - Both "/" & "|" are now replaced with "," so that we have a single delimiter everywhere. 

splitRDD = myRDD.map(lambda str : str.replace("|", ",")).map(lambda str : str.replace("/", ",")).map(lambda str: str.split(","))
splitRDD.take(100)

#raise NotImplementedError()

Out[88]: [['2014-03-15:10:10:20',
 'Sorrento F41L',
 '8cc3b47e-bd01-4482-b500-28f2342679af',
 '7',
 '24',
 '39',
 'enabled',
 'disabled',
 'connected',
 '55',
 '67',
 '12',
 '33.6894754264',
 '-117.543308253'],
 ['2014-03-15:10:10:20',
 'MeeToo 1.0',
 'ef8c7564-0a1a-4650-a655-c8bbd5f8f943',
 '0',
 '31',
 '63',
 '70',
 '39',
 '27',
 'enabled',
 'enabled',
 'enabled',
 '37.4321088904',
 '-121.485029632'],
 ['2014-03-15:10:10:20',
 'MeeToo 1.0',
 '23eba027-b95a-4729-9a4b-a3cca51c5548',
 '0',
 '20',
 '21',
 '86',
 '54',
 '34',
 'enabled',
 'enabled',
 'enabled',
 '39.4378908349',
 '-120.938978486'],
 ['2014-03-15:10:10:20',
 'Sorrento F41L',
 '707daba1-5640-4d60-a6d9-1d6fa0645be0',
 '8',
 '22',
 '60',
 'enabled',
 'enabled',
 'disabled',
 '68',
 '91',
 '17',
 '39.3635186767',
 '-119.400334708'],
 ['2014-03-15:10:10:20',
 'Ronin Novelty Note 1',
 'db66fe81-aa55-43b4-9418-fc6e7a00f891',
 '0',
 '13',
 '47',
 '70',
 'enabled',
 'enabled',
 'enabled',
 '10',
 '45',
 '33.1913581092',
 '-116.448242643'],
 ['2014-03-15:10:10:20',
 'Sorrento F41L',
 'ffa18088-69a0-433e-84b8-006b2b9cc1d0',
 '3',
 '10',
 '36',
 'enabled',
 'connected',
 'enabled',
 '53',
 '58',
 '42',
 '33.8343543748',
 '-117.330000857'],
 ['2014-03-15:10:10:20',
 'Sorrento F33L',
 '66d678e6-9c87-48d2-a415-8d5035e54a23',
 '1',
 '34',
 '74',
 'enabled',
 'disabled',
 'enabled',
 '57',
 '42',
 '15',
 '37.3803954321',
 '-121.840756755'],
 ['2014-03-15:10:10:20',
 'MeeToo 4.1',
 '673f7e4b-d52b-44fc-8826-aea460c3481a',
 '1',
 '16',
 '77',
 '61',
 '24',
 '50',
 'enabled',
 'disabled',
 'enabled',
 '34.1841062345',
 '-117.9435329'],
 ['2014-03-15:10:10:20',
 'Ronin Novelty Note 2',
 'a678ccc3-b0d2-452d-bf89-85bd095e28ee',
 '0',
 '10',
 '97',
 '63',
 'enabled',
 'enabled',
 'connected',
 '48',
 '4',
 '32.2850556785',
 '-111.819583734'],
 ['2014-03-15:10:10:20',
 'Sorrento F41L',
 '86bef6ae-2f1c-42ec-aa67-6acecd7b0675',
 '9',
 '27',
 '35',
 'enabled',
 'enabled',
 'enabled',
 '62',
 '41',
 '19',
 '45.2400522984',
 '-122.377467861'],
 ['2014-03-15:10:10:20',
 'iFruit 3',
 '27178d24-3a61-42f7-a784-e3263f25cc6f',
 '1',
 '30',
 '91',
 'enabled',
 'enabled',
 'enabled',
 '89',
 '41',
 '17',
 '37.9248961741',
 '-122.206868167'],
 ['2014-03-15:10:10:20',
 'Titanic 2400',
 'b4a15931-9a69-469f-9823-a45974472c51',
 '21',
 '96',
 '63',
 '38',
 '11',
 '0',
 'enabled',
 'disabled',
 'enabled',
 '38.1653163975',
 '-122.151608378'],
 ['2014-03-15:10:10:20',
 'Ronin S1',
 'e75dc777-b531-4dbd-80d5-39c772666e6a',
 '0',
 '10',
 '20',
 '47',
 'enabled',
 'connected',
 'enabled',
 '19',
 '36',
 '33.323126641',
 '-116.472234745'],
 ['2014-03-15:10:10:20',
 'Ronin Novelty Note 3',
 'd4ebd9ae-4dad-4fb4-ba1d-d2a9610a458d',
 '0',
 '13',
 '97',
 '63',
 'enabled',
 'enabled',
 'enabled',
 '41',
 '36',
 '33.1774985363',
 '-116.889226299'],
 ['2014-03-15:10:10:20',
 'Ronin Novelty Note 1',
 'b954db08-1f97-4311-8d42-1a7ba39d8dcf',
 '0',
 '23',
 '81',
 '92',
 'enabled',
 'enabled',
 'enabled',
 '4',
 '12',
 '32.2083493316',
 '-111.434102713'],
 ['2014-03-15:10:10:20',
 'MeeToo 1.0',
 '16085fbf-cda5-4489-84b9-0fad888f9e7a',
 '0',
 '29',
 '26',
 '97',
 '8',
 '11',
 'enabled',
 'enabled',
 'enabled',
 '34.0487620041',
 '-111.928871717'],
 ['2014-03-15:10:10:20',
 'iFruit 1',
 '6474caf1-7bbf-4594-a526-9ba8ea82e151',
 '0',
 '15',
 '71',
 'enabled',
 'enabled',
 'connected',
 '77',
 '0',
 '40',
 '37.9031053656',
 '-121.561451342'],
 ['2014-03-15:10:10:20',
 'MeeToo 5.0',
 '668e6f06-a8aa-4be5-8609-899c45d3caa8',
 '0',
 '11',
 '61',
 '71',
 '45',
 '66',
 'enabled',
 'enabled',
 'connected',
 '36.032967794',
 '-118.970108886'],
 ['2014-03-15:10:10:20',
 'Ronin Novelty Note 1',
 '6d195272-8dba-42d6-9b1f-fe61edf7f2a2',
 '0',
 '13',
 '49',
 '98',
 'enabled',
 'connected',
 'enabled',
 '60',
 '16',
 '45.0400810371',
 '-117.858004521'],
 ['2014-03-15:10:10:20',
 'Sorrento F10L',
 'd228cdab-8b35-4733-9f2e-e4760dfacb30',
 '1',
 '22',
 '46',
 'enabled',
 'enabled',
 'enabled',
 '64',
 '59',
 '15',
 '35.2338863976',
 '-114.3057523']

In [0]:
# DO NOT CHANGE THE CONTENT OF THIS CELL
assert splitRDD.take(2) == [[u'2014-03-15:10:10:20',
  u'Sorrento F41L',
  u'8cc3b47e-bd01-4482-b500-28f2342679af',
  u'7',
  u'24',
  u'39',
  u'enabled',
  u'disabled',
  u'connected',
  u'55',
  u'67',
  u'12',
  u'33.6894754264',
  u'-117.543308253'],
 [u'2014-03-15:10:10:20',
  u'MeeToo 1.0',
  u'ef8c7564-0a1a-4650-a655-c8bbd5f8f943',
  u'0',
  u'31',
  u'63',
  u'70',
  u'39',
  u'27',
  u'enabled',
  u'enabled',
  u'enabled',
  u'37.4321088904',
  u'-121.485029632']], "Unexpected entries in the first couple of lines"

Filter out any records which do not parse correctly. The records which do have the correct number of fields should be placed in an RDD named `filteredRDD`.

In [0]:
# The format of your answer should be

# PROBLEM - Total number of elements to check in SplitRDD == 14
# ACTION TAKEN - Filtered only the lists with length = 14, & Checked for the total count and compared with the count of myRDD

filteredRDD = splitRDD.filter(lambda list : len(list)==14)                  
filteredRDD.count()

# Conclusion - All the lists parsed correctly

#raise NotImplementedError()

Out[90]: 459540

In [0]:
# DO NOT CHANGE THE CONTENT OF THIS CELL
assert filteredRDD.count() == 459540, "You may have forgotten to account for some delimiters"

Extract the date (first field), model (second field), device ID (third field), latitude and longitude (13th and 14th fields respectively). Each record's list should therefore reduced to a list of the 5 fields only. The output should be placed in a variable named `extractedRDD`.

In [0]:
# The format of your solution should be

extractedRDD = filteredRDD.map(lambda val: [val[0], val[1], val[2], val[12], val[13]])
extractedRDD.take(20)

#raise NotImplementedError()

Out[92]: [['2014-03-15:10:10:20',
 'Sorrento F41L',
 '8cc3b47e-bd01-4482-b500-28f2342679af',
 '33.6894754264',
 '-117.543308253'],
 ['2014-03-15:10:10:20',
 'MeeToo 1.0',
 'ef8c7564-0a1a-4650-a655-c8bbd5f8f943',
 '37.4321088904',
 '-121.485029632'],
 ['2014-03-15:10:10:20',
 'MeeToo 1.0',
 '23eba027-b95a-4729-9a4b-a3cca51c5548',
 '39.4378908349',
 '-120.938978486'],
 ['2014-03-15:10:10:20',
 'Sorrento F41L',
 '707daba1-5640-4d60-a6d9-1d6fa0645be0',
 '39.3635186767',
 '-119.400334708'],
 ['2014-03-15:10:10:20',
 'Ronin Novelty Note 1',
 'db66fe81-aa55-43b4-9418-fc6e7a00f891',
 '33.1913581092',
 '-116.448242643'],
 ['2014-03-15:10:10:20',
 'Sorrento F41L',
 'ffa18088-69a0-433e-84b8-006b2b9cc1d0',
 '33.8343543748',
 '-117.330000857'],
 ['2014-03-15:10:10:20',
 'Sorrento F33L',
 '66d678e6-9c87-48d2-a415-8d5035e54a23',
 '37.3803954321',
 '-121.840756755'],
 ['2014-03-15:10:10:20',
 'MeeToo 4.1',
 '673f7e4b-d52b-44fc-8826-aea460c3481a',
 '34.1841062345',
 '-117.9435329'],
 ['2014-03-15:10:10:20',
 'Ronin Novelty Note 2',
 'a678ccc3-b0d2-452d-bf89-85bd095e28ee',
 '32.2850556785',
 '-111.819583734'],
 ['2014-03-15:10:10:20',
 'Sorrento F41L',
 '86bef6ae-2f1c-42ec-aa67-6acecd7b0675',
 '45.2400522984',
 '-122.377467861'],
 ['2014-03-15:10:10:20',
 'iFruit 3',
 '27178d24-3a61-42f7-a784-e3263f25cc6f',
 '37.9248961741',
 '-122.206868167'],
 ['2014-03-15:10:10:20',
 'Titanic 2400',
 'b4a15931-9a69-469f-9823-a45974472c51',
 '38.1653163975',
 '-122.151608378'],
 ['2014-03-15:10:10:20',
 'Ronin S1',
 'e75dc777-b531-4dbd-80d5-39c772666e6a',
 '33.323126641',
 '-116.472234745'],
 ['2014-03-15:10:10:20',
 'Ronin Novelty Note 3',
 'd4ebd9ae-4dad-4fb4-ba1d-d2a9610a458d',
 '33.1774985363',
 '-116.889226299'],
 ['2014-03-15:10:10:20',
 'Ronin Novelty Note 1',
 'b954db08-1f97-4311-8d42-1a7ba39d8dcf',
 '32.2083493316',
 '-111.434102713'],
 ['2014-03-15:10:10:20',
 'MeeToo 1.0',
 '16085fbf-cda5-4489-84b9-0fad888f9e7a',
 '34.0487620041',
 '-111.928871717'],
 ['2014-03-15:10:10:20',
 'iFruit 1',
 '6474caf1-7bbf-4594-a526-9ba8ea82e151',
 '37.9031053656',
 '-121.561451342'],
 ['2014-03-15:10:10:20',
 'MeeToo 5.0',
 '668e6f06-a8aa-4be5-8609-899c45d3caa8',
 '36.032967794',
 '-118.970108886'],
 ['2014-03-15:10:10:20',
 'Ronin Novelty Note 1',
 '6d195272-8dba-42d6-9b1f-fe61edf7f2a2',
 '45.0400810371',
 '-117.858004521'],
 ['2014-03-15:10:10:20',
 'Sorrento F10L',
 'd228cdab-8b35-4733-9f2e-e4760dfacb30',
 '35.2338863976',
 '-114.3057523']]

In [0]:
# DO NOT CHANGE THE CONTENT OF THIS CELL
assert extractedRDD.take(2) == [[u'2014-03-15:10:10:20',
  u'Sorrento F41L',
  u'8cc3b47e-bd01-4482-b500-28f2342679af',
  u'33.6894754264',
  u'-117.543308253'],
 [u'2014-03-15:10:10:20',
  u'MeeToo 1.0',
  u'ef8c7564-0a1a-4650-a655-c8bbd5f8f943',
  u'37.4321088904',
  u'-121.485029632']], "There seems to be some discrepancy between your answer and the expected answer"

The second field contains the device manufacturer and model name (e.g. Ronin S2). Split this field by spaces to separate the manufacturer from the model (i.e. manufacturer Ronin, model S2). Each resulting record should therefore now contain a list containing: date, manufacturer, model, device ID, latitude and longitude. The new RDD should be placed in a variable named `separatedRDD`.

In [0]:
# The format of your solution should be

separatedRDD = extractedRDD.map(lambda list: [list[0],list[1].split()[0],list[1].split()[1],list[2],list[3],list[4]])
separatedRDD.take(5)

#raise NotImplementedError()

Out[94]: [['2014-03-15:10:10:20',
 'Sorrento',
 'F41L',
 '8cc3b47e-bd01-4482-b500-28f2342679af',
 '33.6894754264',
 '-117.543308253'],
 ['2014-03-15:10:10:20',
 'MeeToo',
 '1.0',
 'ef8c7564-0a1a-4650-a655-c8bbd5f8f943',
 '37.4321088904',
 '-121.485029632'],
 ['2014-03-15:10:10:20',
 'MeeToo',
 '1.0',
 '23eba027-b95a-4729-9a4b-a3cca51c5548',
 '39.4378908349',
 '-120.938978486'],
 ['2014-03-15:10:10:20',
 'Sorrento',
 'F41L',
 '707daba1-5640-4d60-a6d9-1d6fa0645be0',
 '39.3635186767',
 '-119.400334708'],
 ['2014-03-15:10:10:20',
 'Ronin',
 'Novelty',
 'db66fe81-aa55-43b4-9418-fc6e7a00f891',
 '33.1913581092',
 '-116.448242643']]

In [0]:
# DO NOT CHANGE THE CONTENT OF THIS CELL
assert separatedRDD.take(2) == [[u'2014-03-15:10:10:20',
  u'Sorrento',
  u'F41L',
  u'8cc3b47e-bd01-4482-b500-28f2342679af',
  u'33.6894754264',
  u'-117.543308253'],
 [u'2014-03-15:10:10:20',
  u'MeeToo',
  u'1.0',
  u'ef8c7564-0a1a-4650-a655-c8bbd5f8f943',
  u'37.4321088904',
  u'-121.485029632']], "There is some discrepancy between your answer and the expected answer"

# Remove the output directory for the following section if it already exists
try:
    try:
        imp.find_module('dbutils')
        dbutils.fs.ls("/FileStore/devicestatus_etl/")
        dbutils.fs.rm("/FileStore/devicestatus_etl", True)
    except ImportError:
        import os
        if os.path.exists("/FileStore/devicestatus_etl/"):
            import shutil
            shutil.rmtree("/FileStore/devicestatus_etl/")
except Exception:
    pass

Save the extracted data to comma delimited text files in the `/FileStore/devicestatus_etl` directory on DBFS.

In [0]:
# separatedRDD.saveAsTextFile("dbfs:/FileStore/devicestatus_etl/")

separatedRDD.map(lambda v: ','.join(val for val in v)).saveAsTextFile('/FileStore/devicestatus_etl')

#raise NotImplementedError()

In [0]:
# Checking the folder after saving 
dbutils.fs.ls("dbfs:/FileStore/devicestatus_etl/")

Out[97]: [FileInfo(path='dbfs:/FileStore/devicestatus_etl/_SUCCESS', name='_SUCCESS', size=0),
 FileInfo(path='dbfs:/FileStore/devicestatus_etl/part-00000', name='part-00000', size=22281863),
 FileInfo(path='dbfs:/FileStore/devicestatus_etl/part-00001', name='part-00001', size=22278372)]

In [0]:
# Checking the contents of the file
dbutils.fs.head("dbfs:/FileStore/devicestatus_etl/part-00000", 99)

[Truncated to first 99 bytes]
Out[98]: '2014-03-15:10:10:20,Sorrento,F41L,8cc3b47e-bd01-4482-b500-28f2342679af,33.6894754264,-117.543308253'

In [0]:
# DO NOT CHANGE THE CONTENT OF THIS CELL
try:
    imp.find_module('dbutils')
    assert dbutils.fs.ls("/FileStore/devicestatus_etl"), "You don't appear to have created the required output directory"
    assert dbutils.fs.ls("/FileStore/devicestatus_etl/part-00000"), "The output doesn't appear to be as expected"
    assert dbutils.fs.head("/FileStore/devicestatus_etl/part-00000", 99) == "2014-03-15:10:10:20,Sorrento,F41L,8cc3b47e-bd01-4482-b500-28f2342679af,33.6894754264,-117.543308253", "Expecting different output to that produced"
except ImportError:
    assert os.path.exists("/FileStore/devicestatus_etl"), "You don't appear to have created the required output directory"
    assert os.path.exists("/FileStore/devicestatus_etl/part-00000"), "The output doesn't appear to be as expected"
    
    # Check contents of the first line
    with open("/FileStore/devicestatus_etl/part-00000") as f:
        first_line = f.readline().strip()
        assert first_line == "2014-03-15:10:10:20,Sorrento,F41L,8cc3b47e-bd01-4482-b500-28f2342679af,33.6894754264,-117.543308253", "Expecting different output to that produced"
    
# Count the number of lines in the second part
myRDD = sc.textFile("/FileStore/devicestatus_etl/part-00001")
assert myRDD.count() == 229802, "Unexpected number of lines in part-00001"

[Truncated to first 99 bytes]